### **Lấy và làm sạch Dữ liệu AQI và Weather từ AQI**

![Ảnh](./images/pam1_JLLG.png)

#### **I. Import các thư viện**

In [1]:
import requests
import json
import pandas as pd
import numpy as np
import datetime
from datetime import datetime, timedelta
from time import sleep
%matplotlib inline      

#### **II. Lấy dữ liệu từ API và tạo Dataframe từ data lấy được**
Trong dự án này, dữ liệu chất lượng không khí được thu thập thông qua [Weatherbit API](https://www.weatherbit.io/), một nền tảng tổng hợp dữ liệu khí tượng – môi trường toàn cầu.Weatherbit không trực tiếp đặt cảm biến tại từng vị trí địa lý, mà tổng hợp dữ liệu từ nhiều nguồn khác nhau, bao gồm:
- Các trạm quan trắc mặt đất (ground-based stations),

- Dữ liệu vệ tinh khí tượng (NASA MODIS, TROPOMI, MOPITT, v.v.),

- Và các mô hình khí tượng – hóa học (numerical models) như WRF-Chem, CAMS, GEOS-Chem.

Dữ liệu sau khi được thu thập sẽ được hiệu chỉnh sai số, nội suy không gian và chuẩn hóa theo từng ô lưới (grid) có kích thước khoảng 10 km × 10 km.
Điều này có nghĩa là các khu vực nằm trong cùng một ô lưới (ví dụ các quận nội thành Hà Nội nằm gần nhau) sẽ nhận được cùng một giá trị AQI và nồng độ các chất ô nhiễm (PM₂.₅, PM₁₀, CO, NO₂, SO₂, O₃).

Vì vậy, để tránh hiện tượng dữ liệu bị trùng lặp (nhân bản) giữa các quận lân cận, dự án lựa chọn tọa độ trung tâm của quận Hoàn Kiếm (21.0285°N, 105.8542°E) làm điểm đại diện cho khu vực nội thành Hà Nội. Hoàn Kiếm là khu vực trung tâm thủ đô, có mật độ dân cư cao, nhiều hoạt động giao thông và thương mại, do đó phản ánh tương đối chính xác chất lượng không khí trung bình của toàn khu vực đô thị Hà Nội.


In [2]:
# API_KEY = "d0abdba555a24c308b658ff1a9af5267"
API_KEY = "23c63a63d23b4e18bc2902d841b53ce2"
LAT, LON = 21.0285, 105.8542


start_date = datetime(2025, 12, 4)
end_date = datetime(2025, 12, 31)

urls_air = []
urls_wea = []
current = start_date
while current < end_date:
    next_month = (current.replace(day=28) + timedelta(days=4)).replace(day=1)
    start_str = current.strftime('%Y-%m-%d')
    end_str = next_month.strftime('%Y-%m-%d')

    url_air = f"https://api.weatherbit.io/v2.0/history/airquality?lat={LAT}&lon={LON}&start_date={start_str}&end_date={end_str}&tz=local&key={API_KEY}"
    url_wea = f"https://api.weatherbit.io/v2.0/history/hourly?lat={LAT}&lon={LON}&start_date={start_str}&end_date={end_str}&tz=local&key={API_KEY}"
    urls_air.append(url_air)
    urls_wea.append(url_wea)
    current = next_month

print(f" Tạo {len(urls_air)} URLs ({urls_air[0]} → {urls_air[-1]})")
print(f" Tạo {len(urls_wea)} URLs ({urls_wea[0]} → {urls_wea[-1]})")


 Tạo 1 URLs (https://api.weatherbit.io/v2.0/history/airquality?lat=21.0285&lon=105.8542&start_date=2025-12-04&end_date=2026-01-01&tz=local&key=23c63a63d23b4e18bc2902d841b53ce2 → https://api.weatherbit.io/v2.0/history/airquality?lat=21.0285&lon=105.8542&start_date=2025-12-04&end_date=2026-01-01&tz=local&key=23c63a63d23b4e18bc2902d841b53ce2)
 Tạo 1 URLs (https://api.weatherbit.io/v2.0/history/hourly?lat=21.0285&lon=105.8542&start_date=2025-12-04&end_date=2026-01-01&tz=local&key=23c63a63d23b4e18bc2902d841b53ce2 → https://api.weatherbit.io/v2.0/history/hourly?lat=21.0285&lon=105.8542&start_date=2025-12-04&end_date=2026-01-01&tz=local&key=23c63a63d23b4e18bc2902d841b53ce2)


##### **2.1 AIR QUALITY**

In [3]:
results_air = []
for i, url in enumerate(urls_air):
    print(f'Lấy dữ liệu từ URL {i}/{len(urls_air)} : {url}')
    try:
        renponse = requests.get(url, timeout=30)
        renponse.raise_for_status()
        data = json.loads(renponse.text)
        results_air.append(data)
        sleep(1.2)

    except Exception as e:
        print(f"Lỗi khi lấy dữ liệu {i} : {e}")

print(f"\n Hoàn tất tải {len(results_air)} ")

Lấy dữ liệu từ URL 0/1 : https://api.weatherbit.io/v2.0/history/airquality?lat=21.0285&lon=105.8542&start_date=2025-12-04&end_date=2026-01-01&tz=local&key=23c63a63d23b4e18bc2902d841b53ce2

 Hoàn tất tải 1 


In [4]:
results_air[0]['city_name']

'Hoàn Kiếm'

In [5]:
results_air[0]['data'][0]

{'aqi': 137,
 'co': 605,
 'datetime': '2025-12-31:17',
 'no2': 8,
 'o3': 7.6,
 'pm10': 49,
 'pm25': 49,
 'so2': 32,
 'timestamp_local': '2026-01-01T00:00:00',
 'timestamp_utc': '2025-12-31T17:00:00',
 'ts': 1767200400}

In [6]:
joined_data = []
for res in results_air:
    if 'data' in res:
        joined_data.extend(res['data'])

joined_results = {
    'city_name': results_air[0]['city_name'],
    'country_code': results_air[0]['country_code'],
    'lat': results_air[0]['lat'],
    'lon': results_air[0]['lon'],
    'timezone': results_air[0]['timezone'],
    'data': joined_data
}


In [7]:
joined_results['data'][0]

{'aqi': 137,
 'co': 605,
 'datetime': '2025-12-31:17',
 'no2': 8,
 'o3': 7.6,
 'pm10': 49,
 'pm25': 49,
 'so2': 32,
 'timestamp_local': '2026-01-01T00:00:00',
 'timestamp_utc': '2025-12-31T17:00:00',
 'ts': 1767200400}

In [8]:
joined_results['data'][-1]

{'aqi': 137,
 'co': 605,
 'datetime': '2025-12-31:17',
 'no2': 8,
 'o3': 7.6,
 'pm10': 49,
 'pm25': 49,
 'so2': 32,
 'timestamp_local': '2026-01-01T00:00:00',
 'timestamp_utc': '2025-12-31T17:00:00',
 'ts': 1767200400}

In [9]:

df = pd.DataFrame(joined_results)

df.columns = ['City', 'Country code', 'Lat', 'Lon', 'timezone', 'Data']
df[['AQI', 'CO', 'Date Time', 'NO2', 'O3', 'PM10', 'PM25', 'SO2', 'Local Time', 'UTC Time', 'TS']] = pd.DataFrame(df['Data'].tolist())
df.drop(columns=['Data', 'Lat', 'Lon', 'TS', 'Date Time'], inplace=True)
df = df.drop_duplicates()
df = df.sort_values(by='Local Time')
df['Local Time'] = pd.to_datetime(df['Local Time'])
df.set_index('Local Time', inplace=True)
df

,City,Country code,timezone,AQI,CO,NO2,O3,PM10,PM25,SO2,UTC Time
Local Time,,,,,,,,,,,
2025-12-04 00:00:00,Hoàn Kiếm,VN,Asia/Ho_Chi_Minh,155,735.0,31.0,14.5,67.5,59.33,3.0,2025-12-03T17:00:00
2025-12-04 01:00:00,Hoàn Kiếm,VN,Asia/Ho_Chi_Minh,151,655.0,31.5,53.3,66.3,56.25,3.5,2025-12-03T18:00:00
2025-12-04 02:00:00,Hoàn Kiếm,VN,Asia/Ho_Chi_Minh,155,626.0,32.5,48.6,70.3,59.00,4.5,2025-12-03T19:00:00
2025-12-04 03:00:00,Hoàn Kiếm,VN,Asia/Ho_Chi_Minh,165,629.0,21.0,22.6,76.7,67.00,3.5,2025-12-03T20:00:00
2025-12-04 04:00:00,Hoàn Kiếm,VN,Asia/Ho_Chi_Minh,153,713.0,30.5,23.7,77.0,57.50,3.0,2025-12-03T21:00:00
...,...,...,...,...,...,...,...,...,...,...,...
2025-12-31 20:00:00,Hoàn Kiếm,VN,Asia/Ho_Chi_Minh,153,945.0,23.7,20.3,58.0,58.00,12.0,2025-12-31T13:00:00
2025-12-31 21:00:00,Hoàn Kiếm,VN,Asia/Ho_Chi_Minh,157,2391.0,22.3,42.1,64.0,60.60,13.0,2025-12-31T14:00:00
2025-12-31 22:00:00,Hoàn Kiếm,VN,Asia/Ho_Chi_Minh,159,900.0,14.0,34.1,65.3,62.00,11.0,2025-12-31T15:00:00


In [10]:
df.to_csv('air_quality_data_1.csv', index=True)

In [11]:
df = pd.read_csv('air_quality_data_1.csv')
df.shape

(673, 12)

##### **2.2 WEATHER**

In [ ]:
results_wea = []
for i, url in enumerate(urls_wea):
    print(f'Lấy dữ liệu từ URL {i}/{len(urls_wea)} : {url}')
    try:
        renponse = requests.get(url, timeout=30)
        renponse.raise_for_status()
        data = json.loads(renponse.text)
        results_wea.append(data)
        sleep(1.2)

    except Exception as e:
        print(f"Lỗi khi lấy dữ liệu {i} : {e}")

print(f"\n Hoàn tất tải {len(results_wea)} ")

Lấy dữ liệu từ URL 0/1 : https://api.weatherbit.io/v2.0/history/hourly?lat=21.0285&lon=105.8542&start_date=2025-12-04&end_date=2026-01-01&tz=local&key=23c63a63d23b4e18bc2902d841b53ce2


In [ ]:
joined_data = []
for res in results_wea:
    if 'data' in res:
        joined_data.extend(res['data'])

joined_results_weather = {
    'city_name': results_wea[0]['city_name'],
    'country_code': results_wea[0]['country_code'],
    'lat': results_wea[0]['lat'],
    'lon': results_wea[0]['lon'],
    'timezone': results_wea[0]['timezone'],
    'data': joined_data
}

In [ ]:

df_weather = pd.DataFrame(joined_results_weather)
df_weather.columns = ['City', 'Country code', 'Lat', 'Lon', 'timezone', 'Data']
d = pd.json_normalize(df_weather.pop('Data'))
keep = d[["clouds","precip","pres","rh","temp","uv","wind_spd","timestamp_local","timestamp_utc"]].rename(columns={
    "clouds":"Clouds",
    "precip":"Precipitation",
    "pres":"Pressure",
    "rh":"Relative Humidity",
    "temp":"Temperature",
    "uv":"UV Index",
    "wind_spd":"Wind Speed",
    "timestamp_local":"Local Time",
    "timestamp_utc":"UTC Time"
})
df_weather = pd.concat([df_weather[['City', 'Country code', 'timezone']], keep], axis=1)
df_weather.head()

,City,Country code,timezone,Clouds,Precipitation,Pressure,Relative Humidity,Temperature,UV Index,Wind Speed,Local Time,UTC Time
0,Hoàn Kiếm,VN,Asia/Ho_Chi_Minh,100,0.0,1019,74,18.9,0.0,1.62,2025-12-04T00:00:00,2025-12-03T17:00:00
1,Hoàn Kiếm,VN,Asia/Ho_Chi_Minh,87,0.0,1018,74,18.5,0.0,1.55,2025-12-04T01:00:00,2025-12-03T18:00:00
2,Hoàn Kiếm,VN,Asia/Ho_Chi_Minh,80,0.0,1018,75,18.2,0.0,1.38,2025-12-04T02:00:00,2025-12-03T19:00:00
3,Hoàn Kiếm,VN,Asia/Ho_Chi_Minh,68,0.0,1018,76,17.8,0.0,1.32,2025-12-04T03:00:00,2025-12-03T20:00:00
4,Hoàn Kiếm,VN,Asia/Ho_Chi_Minh,75,0.0,1018,75,18.6,0.0,1.01,2025-12-04T04:00:00,2025-12-03T21:00:00


In [ ]:
df_weather.to_csv('weather_data_1.csv', index=False)

In [ ]:
df_weather = pd.read_csv('weather_data_1.csv')
# df_weather.info()


#### **IV. Hợp nhất hai khung dữ liệu và sắp xếp dữ liệu**

In [ ]:

merged_df = pd.merge(df, df_weather, left_index=True, right_index=True)

merged_df.drop(columns=['City_y', 'Country code_y', 'timezone_y', 'UTC Time_y'], inplace=True)

utc_time_column = merged_df.pop('UTC Time_x')
merged_df.insert(0, 'UTC Time', utc_time_column)

merged_df = merged_df.rename(columns={'City_x': 'City', 'Country code_x': 'Country Code', 'timezone_x':'Timezone'})
merged_df

,UTC Time,Local Time_x,City,Country Code,Timezone,AQI,CO,NO2,O3,PM10,PM25,SO2,Clouds,Precipitation,Pressure,Relative Humidity,Temperature,UV Index,Wind Speed,Local Time_y
0,2025-12-03T17:00:00,2025-12-04 00:00:00,Hoàn Kiếm,VN,Asia/Ho_Chi_Minh,155,735.0,31.0,14.5,67.5,59.33,3.0,100,0.0,1019,74,18.9,0.0,1.62,2025-12-04T00:00:00
1,2025-12-03T18:00:00,2025-12-04 01:00:00,Hoàn Kiếm,VN,Asia/Ho_Chi_Minh,151,655.0,31.5,53.3,66.3,56.25,3.5,87,0.0,1018,74,18.5,0.0,1.55,2025-12-04T01:00:00
2,2025-12-03T19:00:00,2025-12-04 02:00:00,Hoàn Kiếm,VN,Asia/Ho_Chi_Minh,155,626.0,32.5,48.6,70.3,59.00,4.5,80,0.0,1018,75,18.2,0.0,1.38,2025-12-04T02:00:00
3,2025-12-03T20:00:00,2025-12-04 03:00:00,Hoàn Kiếm,VN,Asia/Ho_Chi_Minh,165,629.0,21.0,22.6,76.7,67.00,3.5,68,0.0,1018,76,17.8,0.0,1.32,2025-12-04T03:00:00
4,2025-12-03T21:00:00,2025-12-04 04:00:00,Hoàn Kiếm,VN,Asia/Ho_Chi_Minh,153,713.0,30.5,23.7,77.0,57.50,3.0,75,0.0,1018,75,18.6,0.0,1.01,2025-12-04T04:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,2025-12-31T12:00:00,2025-12-31 19:00:00,Hoàn Kiếm,VN,Asia/Ho_Chi_Minh,152,1470.5,57.7,22.5,57.0,57.00,39.3,0,0.0,1010,83,21.2,0.0,3.70,2025-12-31T19:00:00
668,2025-12-31T13:00:00,2025-12-31 20:00:00,Hoàn Kiếm,VN,Asia/Ho_Chi_Minh,153,945.0,23.7,20.3,58.0,58.00,12.0,0,0.0,1011,85,20.6,0.0,3.15,2025-12-31T20:00:00
669,2025-12-31T14:00:00,2025-12-31 21:00:00,Hoàn Kiếm,VN,Asia/Ho_Chi_Minh,157,2391.0,22.3,42.1,64.0,60.60,13.0,0,0.0,1011,87,20.1,0.0,2.63,2025-12-31T21:00:00
670,2025-12-31T15:00:00,2025-12-31 22:00:00,Hoàn Kiếm,VN,Asia/Ho_Chi_Minh,159,900.0,14.0,34.1,65.3,62.00,11.0,61,0.0,1012,88,19.8,0.0,2.32,2025-12-31T22:00:00


In [ ]:

merged_df.shape

(672, 20)

In [ ]:
merged_df.rename(columns={'Local Time_x': 'Local Time'}, inplace=True)
merged_df = merged_df[['Local Time', 'UTC Time', 'City', 'Country Code', 'Timezone', 'AQI', 'CO', 'NO2', 'O3', 'PM10', 'PM25', 'SO2',
                       'Clouds', 'Precipitation', 'Pressure', 'Relative Humidity', 'Temperature', 'UV Index', 'Wind Speed']]

merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 672 entries, 0 to 671
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Local Time         672 non-null    object 
 1   UTC Time           672 non-null    object 
 2   City               672 non-null    object 
 3   Country Code       672 non-null    object 
 4   Timezone           672 non-null    object 
 5   AQI                672 non-null    int64  
 6   CO                 672 non-null    float64
 7   NO2                672 non-null    float64
 8   O3                 672 non-null    float64
 9   PM10               672 non-null    float64
 10  PM25               672 non-null    float64
 11  SO2                672 non-null    float64
 12  Clouds             672 non-null    int64  
 13  Precipitation      672 non-null    float64
 14  Pressure           672 non-null    int64  
 15  Relative Humidity  672 non-null    int64  
 16  Temperature        672 non-null

In [ ]:
# Gộp các file lại csv
data22 = pd.read_csv('E:\Document\ĐỒ ÁN 1\Air_Quality-Analysis_HN\data\data21.csv')
data2324 = pd.read_csv('E:\Document\ĐỒ ÁN 1\Air_Quality-Analysis_HN\data\data2324.csv')

data2324 = data2324[data22.columns]
merge_data = pd.concat([data22, data2324])


merge_data.to_csv('E:\Document\ĐỒ ÁN 1\Air_Quality-Analysis_HN\data\data2224.csv', index=False)

In [ ]:
merge_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26009 entries, 0 to 17543
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Local Time         26009 non-null  object 
 1   UTC Time           26009 non-null  object 
 2   City               26009 non-null  object 
 3   Country Code       26009 non-null  object 
 4   Timezone           26009 non-null  object 
 5   AQI                26009 non-null  int64  
 6   CO                 26009 non-null  float64
 7   NO2                26009 non-null  float64
 8   O3                 26009 non-null  float64
 9   PM10               26009 non-null  float64
 10  PM25               26009 non-null  float64
 11  SO2                26009 non-null  float64
 12  Clouds             26009 non-null  int64  
 13  Precipitation      26009 non-null  float64
 14  Pressure           26009 non-null  int64  
 15  Relative Humidity  26009 non-null  int64  
 16  Temperature        26009 no

In [ ]:
merge_data.interpolate(method='linear', limit_direction='forward', inplace=True)

C:\Users\hungd\AppData\Local\Temp\ipykernel_18700\1762461018.py:1: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  merge_data.interpolate(method='linear', limit_direction='forward', inplace=True)


In [ ]:

merged_df.to_csv('data/raw/data_test_2025.csv', index=False)